In [11]:
!git clone https://github.com/gagansh7171/GLUECoS_IJCNLP.git

Cloning into 'GLUECoS_IJCNLP'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (180/180), done.
remote: Total 202 (delta 27), reused 98 (delta 6), pack-reused 0
Receiving objects: 100% (202/202), 8.48 MiB | 5.89 MiB/s, done.
Resolving deltas: 100% (27/27), done.


In [69]:
# !pip3 install transformers
# !pip3 install SentencePiece 
# ! pip3 install wordninja
# !pip3 install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


The dataset is available in results3 branch of the above repository.

In [14]:
!cd GLUECoS_IJCNLP/ ; cp -r . ../

In [15]:
import pandas as pd

def distribution(task, dataset, file):
  df = pd.read_csv(file,header=None, sep='\t', lineterminator ='\n') 
  pos = 0
  neg = 0
  neu = 0
  for i in range(len(df[1])):
    if df[1][i] == "neutral":
      neu+=1
    elif df[1][i] == "negative":
      neg+=1
    else:
      pos += 1
  print(task + "-" +dataset)
  print(" positive = " + str(pos) + "\n negative = "+ str(neg) + "\n neutral = "+str(neu) + "\n total = " + str(pos+neg+neu))
distribution("Sentiment Analysis En-Hi", "Training", "./Data/Processed_Data/Sentiment_EN_HI/Romanized/train.txt")
distribution("Sentiment Analysis En-Hi", "Validation", "./Data/Processed_Data/Sentiment_EN_HI/Romanized/validation.txt")
distribution("Sentiment Analysis En-Es", "Training", "./Data/Processed_Data/Sentiment_EN_ES/train.txt")
distribution("Sentiment Analysis En-Es", "Validation", "./Data/Processed_Data/Sentiment_EN_ES/validation.txt")

Sentiment Analysis En-Hi-Training
 positive = 3202
 negative = 2319
 neutral = 4559
 total = 10080
Sentiment Analysis En-Hi-Validation
 positive = 399
 negative = 283
 neutral = 578
 total = 1260
Sentiment Analysis En-Es-Training
 positive = 433
 negative = 379
 neutral = 711
 total = 1523
Sentiment Analysis En-Es-Validation
 positive = 55
 negative = 46
 neutral = 83
 total = 184


The hyper-parameters for the experiments can be edited in ./Code/train_sentence.sh file

In [20]:
!bash train.sh bert-base-multilingual-cased bert Sentiment_EN_ES

Fine-tuning bert-base-multilingual-cased on Sentiment_EN_ES
2023-05-24 08:30:43.481414: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
bert bert-base-multilingual-cased
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model tha

The results are generated in ./Results folder. The logits for a model is generated as modelname_val_logits.txt and modelnamelogits.txt. Copy these files to the root and run above script again with another model.

In [34]:
!bash train.sh xlm-roberta-base xlm-roberta Sentiment_EN_ES

Fine-tuning xlm-roberta-base on Sentiment_EN_ES
2023-05-24 08:45:59.354536: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
xlm-roberta xlm-roberta-base
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a B

## Prepare the neural network and train it.

In [41]:
import pandas as pd
import os
path = '/content/Results/Sentiment_EN_ES/'
files=['bertlogits.txt', 'xlm-robertalogits.txt']
# for i in files:
train_df = pd.read_csv(files[0], sep='\t', lineterminator='\n')
train_df2 = pd.read_csv(files[1], sep='\t', lineterminator='\n')
a = [train_df, train_df2]
train = pd.concat(a, axis=1)
print(train)
train_y = pd.read_csv('/content/Data/Processed_Data/Sentiment_EN_ES/train.txt',sep='\t', lineterminator='\n', header=None)
train_y = train_y[1]
print(train_y)

      positive  negative   neutral  positive  negative   neutral
0     3.445673 -2.059765 -1.786218  2.583259 -1.564791 -0.692742
1     3.449307 -1.949215 -1.647322  2.582306 -2.354398  0.086241
2    -1.579569  3.367073 -1.036351 -1.156467  1.705231 -0.473242
3    -1.803032  3.532412 -1.045653 -2.474017  2.931765 -0.469971
4    -1.942236 -1.970134  3.301459 -2.076820 -1.311225  3.281563
...        ...       ...       ...       ...       ...       ...
1518 -2.292253 -1.620165  3.402530 -2.149433 -0.440585  2.514155
1519  2.075063 -1.920200 -0.671557  2.423130 -2.249368  0.047057
1520 -2.331498 -1.572748  3.355923 -2.093344 -0.908913  2.909816
1521 -2.068880 -1.870117  3.381201 -1.686527 -0.923312  2.579558
1522 -2.274998 -1.619970  3.409286 -2.098812 -0.585570  2.686048

[1523 rows x 6 columns]
0       positive
1       positive
2       negative
3       negative
4        neutral
          ...   
1518     neutral
1519    positive
1520     neutral
1521     neutral
1522     neutral
Name: 1,

In [42]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer
y = np.array(train_y)
y_dense = LabelBinarizer().fit_transform(y)
y_dense

array([[0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [76]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(12,6), max_iter=200)
train_x = train.to_numpy()
clf.fit(train_x, y_dense)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(12, 6))

Generate logits for the test data set

In [73]:
!bash train2.sh xlm-roberta-base xlm-roberta Sentiment_EN_ES

Fine-tuning xlm-roberta-base on Sentiment_EN_ES
2023-05-24 09:35:30.393368: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification 

The results generated as ./Results/Sentiment_EN_ES/xlm-robertalogits.txt.
Rename the file and move it to the root directory.

In [74]:
!bash train2.sh bert-base-multilingual-cased bert Sentiment_EN_ES

Fine-tuning bert-base-multilingual-cased on Sentiment_EN_ES
2023-05-24 09:39:54.107192: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identic

In [77]:
files=['bertlogits.txt', 'xlm-robertalogits.txt']
# for i in files:
train_df = pd.read_csv(files[0], sep='\t', lineterminator='\n')
train_df2 = pd.read_csv(files[1], sep='\t', lineterminator='\n')
a = [train_df, train_df2]
test = pd.concat(a, axis=1)
print(test)
pred = clf.predict(test.to_numpy())
num=0
with open('test_predictions.txt', 'w') as writer:
  with open('Results/Sentiment_EN_ES/test_predictions.txt') as w:
    for i in range(len(pred)):
      ww = w.readline()
      if not ww:
        break
      if pred[i][0]==1:
        writer.write('negative\n')
      elif pred[i][2]==1:
        writer.write('positive\n')
      else:
        num+=1
        print(pred[i])
        writer.write('neutral\n')
print(num)

     positive  negative   neutral  positive  negative   neutral
0   -2.550151  3.111067 -0.092559 -2.301457  2.595721 -0.173255
1   -2.873883 -0.380576  2.902514 -2.321459  0.081670  2.212453
2   -1.514833 -1.736319  2.796386 -2.075906 -0.804084  2.920364
3   -2.280511  3.290388 -0.478538 -2.298255  2.907766 -0.591298
4   -2.292328 -1.492913  3.345024 -1.399013 -1.385932  2.631538
..        ...       ...       ...       ...       ...       ...
227  0.472178 -1.488765  0.811256 -2.293866  0.055036  2.106927
228  3.095557 -1.595691 -1.604291  2.834644 -2.292464 -0.282852
229 -2.276879 -1.542917  3.475290 -2.792641 -0.325252  2.999851
230 -2.873883 -0.380577  2.902515 -2.321460  0.081672  2.212455
231 -2.818033 -0.447612  2.971394 -2.225531 -0.574466  2.818949

[232 rows x 6 columns]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 0 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]


The results are generated in test_predictions.txt file. To generate the score; zip this file and submit it at https://github.com/microsoft/GLUECoS 